In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport
import torch

In [2]:
alert_df=pd.read_json("/Users/fortes/neuefische/Capstone-WIMT/data/alerts.json")

In [3]:
notification_df=pd.read_json("/Users/fortes/neuefische/Capstone-WIMT/data/notifications.json")

In [4]:
notification_df

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value
0,20210529,1622251203899000,notification_receive,283a1,label,-Maph6wn8GA3SjPIc9HA
1,20210529,1622257988561000,notification_receive,283a1,label,-Maq63vqDUxQRawQ_3Au
2,20210529,1622258235184001,notification_receive,283a1,label,-Maq7-D7bhcEaFBU1LDW
3,20210529,1622260301025000,notification_receive,283a1,label,-MaqEsbKAqlxGG-RupjR
4,20210529,1622293962198000,notification_receive,283a1,label,-MasFHCt_lTuNt_x16dH
...,...,...,...,...,...,...
2278146,20210210,1612909178334000,notification_receive,fff18,label,0NbaiCRr6xGFd2035-Gdlw
2278147,20210210,1612969431222000,notification_receive,fff18,label,EMDjKLFr6xGRbrsa0gDjhg
2278148,20210210,1612978431552000,notification_receive,fff18,label,YE2VDMZr6xGRbrsa0gDjhg
2278149,20210210,1612985245425000,notification_receive,fff18,label,YBu-7NVr6xGRbrsa0gDjhg


In [5]:
#!pip3 install transformers --no-input

from transformers import pipeline

nlp_pos_syntax = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-pos-syntax",
    tokenizer="mrm8488/bert-spanish-cased-finetuned-pos-syntax"
)

text = 'Considera retrasos por servicios de emergencia atendiendo incendio en Mesones y Av. 20 de Noviembre.'

nlp_pos_syntax(text)[1:len(nlp_pos_syntax(text))-1]

In [6]:
cause_effect_table= pd.pivot_table(alert_df, index=['cause'], columns='effect',aggfunc='count',fill_value=0,values="is_global", margins=True, margins_name= 'Total')

In [7]:
cause_effect_table

effect,,CirculationRestored,CirculationShutdown,Delays,Evacuation,FullCapacity,HighWaitingTime,InterimService,LaneReduction,RouteDetour,SecuritySpeed,SuspensionOfService,TrafficImpact,Total
cause,,,,,,,,,,,,,,
BrokenTrafficLight,0,0,0,23,0,0,0,0,2,0,0,0,22,47
BrokenVehicle,0,0,4,390,0,0,0,0,39,0,0,12,55,500
COVID19,0,0,7,1,0,5,0,0,0,0,0,18,1,32
Counterflow,0,0,0,2,0,0,0,0,0,0,0,0,2,4
CycleRide,0,0,4,3,0,0,0,0,0,0,0,0,16,23
Demonstration,1,321,1896,166,0,0,0,65,54,5,6,251,226,2991
EmergencyServices,2,1,20,860,0,0,0,0,208,0,0,4,253,1348
Event,0,2,4,0,0,0,0,0,2,0,3,5,0,16
Explosion,0,0,0,0,0,0,0,0,0,0,0,0,1,1


Dropping the rows that contain source in join_field

In [8]:
source_purge= notification_df[(notification_df["join_field"]=="source")].index
notification_df.drop(source_purge, axis=0, inplace=True)

Fixing the event_name column:

In [9]:
notification_df["event_name"].replace("notification_receive","notification_received", inplace=True)

In [10]:
notification_df["event_name"].replace("notification_open","notification_opened", inplace=True)

In [56]:
merged_df= pd.merge(notification_df, alert_df, left_on="join_key_value", right_on="document_id", how="outer")

In [ ]:
print (notification_df.shape())
print (alert_df.shape())

In [ ]:
merged_df.head()

In [11]:
label_merger= notification_df.query("join_field == 'label'")
alert_id_merger= notification_df.query("join_field == 'alertId'")

In [39]:
label_merger.head()

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value
0,20210529,1622251203899000,notification_received,283a1,label,-Maph6wn8GA3SjPIc9HA
1,20210529,1622257988561000,notification_received,283a1,label,-Maq63vqDUxQRawQ_3Au
2,20210529,1622258235184001,notification_received,283a1,label,-Maq7-D7bhcEaFBU1LDW
3,20210529,1622260301025000,notification_received,283a1,label,-MaqEsbKAqlxGG-RupjR
4,20210529,1622293962198000,notification_received,283a1,label,-MasFHCt_lTuNt_x16dH


In [49]:
merged_df_1= pd.merge(label_merger, alert_df, left_on="join_key_value", right_on="document_id", how="outer")
merged_df_2= pd.merge(alert_id_merger, alert_df, left_on="join_key_value", right_on="document_id", how="outer")

In [47]:
#label_merger.where(label_merger["join_key_value"] == alert_df["surrogate_id"],True, False)
label_merger['join_key_value'].isin(alert_df['document_id']).value_counts()

#dfAns = df1.merge(df2, on = 'id', how = 'outer').merge(df3, on='id', how='outer').fillna(0)

False    2109155
True       93912
Name: join_key_value, dtype: int64

In [51]:
alert_id_merger['join_key_value'].isin(alert_df['document_id']).value_counts()

True     32183
False     4310
Name: join_key_value, dtype: int64

In [54]:
label_merger['join_key_value'].isin(alert_df['surrogate_id']).value_counts()

False    2203067
Name: join_key_value, dtype: int64

In [55]:
alert_id_merger['join_key_value'].isin(alert_df['surrogate_id']).value_counts()

False    36493
Name: join_key_value, dtype: int64

In [37]:
label_merger.shape

(2203067, 6)

In [38]:
alert_id_merger.shape

(36493, 6)

In [25]:
merged_df= pd.concat([merged_df_1, merged_df_2], axis=0)

In [35]:
merged_df_1["agency"].isnull().count()

2203067

In [26]:
print(merged_df.shape[0]- merged_df_1.shape[0]-merged_df_2.shape[0])

0


In [32]:
merged_df_1

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,is_global,agency,area_of_effect_coordinates
0,20210529,1622251203899000,notification_received,283a1,label,-Maph6wn8GA3SjPIc9HA,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,20210529,1622257988561000,notification_received,283a1,label,-Maq63vqDUxQRawQ_3Au,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,20210529,1622258235184001,notification_received,283a1,label,-Maq7-D7bhcEaFBU1LDW,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,20210529,1622260301025000,notification_received,283a1,label,-MaqEsbKAqlxGG-RupjR,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,20210529,1622293962198000,notification_received,283a1,label,-MasFHCt_lTuNt_x16dH,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203062,20210210,1612909178334000,notification_received,fff18,label,0NbaiCRr6xGFd2035-Gdlw,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2203063,20210210,1612969431222000,notification_received,fff18,label,EMDjKLFr6xGRbrsa0gDjhg,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2203064,20210210,1612978431552000,notification_received,fff18,label,YE2VDMZr6xGRbrsa0gDjhg,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2203065,20210210,1612985245425000,notification_received,fff18,label,YBu-7NVr6xGRbrsa0gDjhg,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
merged_df_2

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,is_global,agency,area_of_effect_coordinates
0,20210529,1622254038809000,notification_received,0f737,alertId,-Maps-egC8fFLZM4HR5g,-Maps-egC8fFLZM4HR5g,wB4ikyLA6xGpKOFIXMTiIg,2021-05-29 02:07:09+00:00,2021-05-29 02:07:12+00:00,2021-05-29 03:01:00+00:00,VehicularAccident,Delays,⌛ ¡No de nuevo! Hay retrasos por accidente ve...,false,JfA8Bw8Zp024Kqu4AXiSpQ,"[{'latitude': -99.17095221364039, 'longitude':..."
1,20210529,1622260302438000,notification_received,0f737,alertId,-MaqEsbKAqlxGG-RupjR,-MaqEsbKAqlxGG-RupjR,UFB5JTHA6xGpKOFIXMTiIg,2021-05-29 03:51:27+00:00,2021-05-29 03:51:33+00:00,2021-05-29 04:37:00+00:00,Demonstration,CirculationRestored,✋ ¡Mucho ojo! Se restablece la circulación de ...,false,JfA8Bw8Zp024Kqu4AXiSpQ,"[{'latitude': -99.15240583806413, 'longitude':..."
2,20210529,1622260302438000,notification_received,0f737,alertId,-MaqEsbKAqlxGG-RupjR,-MaqEsbKAqlxGG-RupjR,UFB5JTHA6xGpKOFIXMTiIg,2021-05-29 03:51:27+00:00,2021-05-29 03:51:33+00:00,2021-05-29 04:37:00+00:00,Demonstration,CirculationRestored,✋ ¡Mucho ojo! Se restablece la circulación de ...,false,jLjibFoim0iwWau4AWoEdQ,"[{'latitude': -99.15240583806413, 'longitude':..."
3,20210529,1622260302438000,notification_received,0f737,alertId,-MaqEsbKAqlxGG-RupjR,-MaqEsbKAqlxGG-RupjR,UFB5JTHA6xGpKOFIXMTiIg,2021-05-29 03:51:27+00:00,2021-05-29 03:51:33+00:00,2021-05-29 04:37:00+00:00,Demonstration,CirculationRestored,✋ ¡Mucho ojo! Se restablece la circulación de ...,false,zCy9zG00HEqGeKu4AWZYNQ,"[{'latitude': -99.15240583806413, 'longitude':..."
4,20210529,1622254036776000,notification_received,baafe,alertId,-Maps-egC8fFLZM4HR5g,-Maps-egC8fFLZM4HR5g,wB4ikyLA6xGpKOFIXMTiIg,2021-05-29 02:07:09+00:00,2021-05-29 02:07:12+00:00,2021-05-29 03:01:00+00:00,VehicularAccident,Delays,⌛ ¡No de nuevo! Hay retrasos por accidente ve...,false,JfA8Bw8Zp024Kqu4AXiSpQ,"[{'latitude': -99.17095221364039, 'longitude':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50040,20210706,1625581262819000,notification_received,e0b4d,alertId,-MdwBMI8mNXadCw7GwTc,-MdwBMI8mNXadCw7GwTc,kCtjX2Xe6xGa3QeEZ-Zg-g,2021-07-06 14:20:53+00:00,2021-07-06 14:20:59+00:00,2021-07-06 15:21:33+00:00,Demonstration,SuspensionOfService,🚫 ¡Que no te agarre en curva! Hay suspensión d...,false,JfA8Bw8Zp024Kqu4AXiSpQ,[{}]
50041,20210706,1625584900674000,notification_received,e0b4d,alertId,-MdwPCoa1U3j5wMZ8J99,-MdwPCoa1U3j5wMZ8J99,YNy_023e6xGa3QeEZ-Zg-g,2021-07-06 15:21:25+00:00,2021-07-06 15:21:36+00:00,2021-07-06 20:46:48+00:00,Demonstration,SuspensionOfService,🚫 ¡Ah jijo! Hay suspensión del servicio por ma...,false,JfA8Bw8Zp024Kqu4AXiSpQ,[{}]
50042,20210706,1625587521801000,notification_received,e0b4d,alertId,-MdwZE8hMtqpcYoaNhzM,-MdwZE8hMtqpcYoaNhzM,0AF_8XPe6xGa3QeEZ-Zg-g,2021-07-06 16:05:11+00:00,2021-07-06 16:05:16+00:00,2021-07-06 17:37:59+00:00,Demonstration,SuspensionOfService,🚫 ¡Changos! Hay suspensión del servicio por ma...,false,JfA8Bw8Zp024Kqu4AXiSpQ,[{}]
50043,20210706,1625528529057001,notification_received,427c1,alertId,-Mdt2BT-2y05YXyJctt2,-Mdt2BT-2y05YXyJctt2,ADiylurd6xGf17VsriRlgg,2021-07-05 23:41:58+00:00,2021-07-05 23:42:01+00:00,2021-07-06 02:00:01+00:00,Rain,Delays,⏳ ¿Un cafecito con pan o qué? Debido a lluvias...,false,JUR9bFXmVkWDHqu4AXaY0g,[{}]


In [17]:
merged_df

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,is_global,agency,area_of_effect_coordinates
0,20210529,1622251203899000,notification_received,283a1,label,-Maph6wn8GA3SjPIc9HA,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,20210529,1622257988561000,notification_received,283a1,label,-Maq63vqDUxQRawQ_3Au,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,20210529,1622258235184001,notification_received,283a1,label,-Maq7-D7bhcEaFBU1LDW,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,20210529,1622260301025000,notification_received,283a1,label,-MaqEsbKAqlxGG-RupjR,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,20210529,1622293962198000,notification_received,283a1,label,-MasFHCt_lTuNt_x16dH,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36488,20210706,1625581262819000,notification_received,e0b4d,alertId,-MdwBMI8mNXadCw7GwTc,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
36489,20210706,1625584900674000,notification_received,e0b4d,alertId,-MdwPCoa1U3j5wMZ8J99,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
36490,20210706,1625587521801000,notification_received,e0b4d,alertId,-MdwZE8hMtqpcYoaNhzM,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
36491,20210706,1625528529057001,notification_received,427c1,alertId,-Mdt2BT-2y05YXyJctt2,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
merged_df_1

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,is_global,agency,area_of_effect_coordinates
0,20210529,1622251203899000,notification_received,283a1,label,-Maph6wn8GA3SjPIc9HA,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,20210529,1622257988561000,notification_received,283a1,label,-Maq63vqDUxQRawQ_3Au,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,20210529,1622258235184001,notification_received,283a1,label,-Maq7-D7bhcEaFBU1LDW,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,20210529,1622260301025000,notification_received,283a1,label,-MaqEsbKAqlxGG-RupjR,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,20210529,1622293962198000,notification_received,283a1,label,-MasFHCt_lTuNt_x16dH,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203062,20210210,1612909178334000,notification_received,fff18,label,0NbaiCRr6xGFd2035-Gdlw,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2203063,20210210,1612969431222000,notification_received,fff18,label,EMDjKLFr6xGRbrsa0gDjhg,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2203064,20210210,1612978431552000,notification_received,fff18,label,YE2VDMZr6xGRbrsa0gDjhg,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2203065,20210210,1612985245425000,notification_received,fff18,label,YBu-7NVr6xGRbrsa0gDjhg,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
merged_df["created_at"].isnull().count()

2239560

In [21]:
merged_df.shape

(2239560, 17)